### Day 16: Ticket Translation

### --- Part One ---

As you're walking to yet another connecting flight, you realize that one of the legs of your re-routed trip coming up is on a high-speed train. However, the train ticket you were given is in a language you don't understand. You should probably figure out what it says before you get to the train station after the next flight.

Unfortunately, you can't actually **read** the words on the ticket. You can, however, read the numbers, and so you figure out **the fields these tickets must have** and **the valid ranges** for values in those fields.

You collect the **rules for ticket fields**, the **numbers on your ticket**, and the **numbers on other nearby tickets** for the same train service (via the airport security cameras) together into a single document you can reference (your puzzle input).

The **rules for ticket fields** specify a list of fields that exist **somewhere** on the ticket and the **valid ranges of values** for each field. For example, a rule like `class: 1-3 or 5-7` means that one of the fields in every ticket is named `class` and can be any value in the ranges `1-3 or 5-7` (inclusive, such that `3` and `5` are both valid in this field, but `4` is not).

Each ticket is represented by a single line of comma-separated values. The values are the numbers on the ticket in the order they appear; every ticket has the same format. For example, consider this ticket:

```
.--------------------------------------------------------.
| ????: 101    ?????: 102   ??????????: 103     ???: 104 |
|                                                        |
| ??: 301  ??: 302             ???????: 303      ??????? |
| ??: 401  ??: 402           ???? ????: 403    ????????? |
'--------------------------------------------------------'
```

Here, `?` represents text in a language you don't understand. This ticket might be represented as `101,102,103,104,301,302,303,401,402,403`; of course, the actual train tickets you're looking at are much **more** complicated. In any case, you've extracted just the numbers in such a way that the first number is always the same specific field, the second number is always a different specific field, and so on - you just don't know what each position actually means!

Start by determining which tickets are **completely invalid**; these are tickets that contain values which **aren't valid for any field**. Ignore **your ticket** for now.

For example, suppose you have the following notes:

```
class: 1-3 or 5-7
row: 6-11 or 33-44
seat: 13-40 or 45-50

your ticket:
7,1,14

nearby tickets:
7,3,47
40,4,50
55,2,20
38,6,12
```

It doesn't matter which position corresponds to which field; you can identify invalid **nearby tickets** by considering only whether tickets contain **values that are not valid for any field**. In this example, the values on the first **nearby ticket** are all valid for at least one field. This is not true of the other three **nearby tickets**: the values `4`, `55`, and `12` are are not valid for any field. Adding together all of the invalid values produces your **ticket scanning error rate: 4 + 55 + 12 = 71**.

Consider the validity of the **nearby tickets** you scanned. **What is your ticket scanning error rate?**

In [1]:
blocks = open('day16.data').read().split('\n\n')
accepted = {}
for line in [_l for _l in blocks[0].split('\n')]:
    """Transforms 2 ranges into a flat list of integer: 1-2 or 4-6 -> [1, 2, 4, 5, 6]"""
    data = line.split(': ')
    accepted[data[0]] = sum(((list(range(*[int(n) + i for i, n in enumerate(_range.split('-'))])))
                             for _range in data[1].split(' or ')), [])

def find_error(ticket: str):
    for number in [int(n) for n in ticket.split(',')]:
        if not any(number in acceptable for acceptable in accepted.values()):
            return number
    return None

tickets = list([(line, find_error(line)) for line in [_l for _l in blocks[2].split('\n')[1:]]])
part_1 = sum([t[1] for t in tickets if t[1]])

print(f"Answer: {part_1}")

Answer: 28884


### --- Part Two ---

Now that you've identified which tickets contain invalid values, **discard those tickets entirely**. Use the remaining valid tickets to determine which field is which.

Using the valid ranges for each field, determine what order the fields appear on the tickets. The order is consistent between all tickets: if seat is the third field, it is the third field on every ticket, including **your ticket**.

For example, suppose you have the following notes:

```
class: 0-1 or 4-19
row: 0-5 or 8-19
seat: 0-13 or 16-19

your ticket:
11,12,13

nearby tickets:
3,9,18
15,1,5
5,14,9
```

Based on the **nearby tickets** in the above example, the first position must be `row`, the second position must be `class`, and the third position must be `seat`; you can conclude that in **your ticket**, `class` is `12`, `row` is `11`, and `seat` is `13`.

Once you work out which field is which, look for the six fields on **your ticket** that start with the word `departure`. **What do you get if you multiply those six values together?**

In [2]:
def part_2():
    """associate all positions to all fields"""
    positions = {k: list([i for i in range(len(accepted))]) for k in accepted}  # initially all is accepted
    for valid in [t[0] for t in tickets if t[1] is None]:
        for i, value in enumerate([int(n) for n in valid.split(',')]):
            [positions[field].remove(i) for field, acceptable in accepted.items() if value not in acceptable]

    """remove positions that do not fit by rule on each field"""
    used_positions = set()
    fields = {}
    for field, values in sorted(positions.items(), key=lambda entry: len(entry[1])):  # sort by number of positions
        for position in [_p for _p in values if _p not in used_positions]:
            fields[position] = field
            used_positions.add(position)

    """multiply just the encountered fields that starts with 'departure'"""
    departure_data = 1
    for position, field in [(_p, _f) for _p, _f in fields.items() if _f.startswith('departure')]:
        departure_data *= next(int(each) for i, each in enumerate(blocks[1].split(',')) if i == position)

    return departure_data

print(f"Answer: {part_2()}")

Answer: 1001849322119
